# Working with Containers

### Introduction

In the previous lesson, we saw that virtual machines provided the benefits of isolated and reproducible environments.  But unfortunately, these environments were not lightweight.  This is because, for each isolated environment, virtual machines used a separate operating system each with it's own kernel.

### New innovations on Linux

Now the Linux operating system has a couple of features that allow for the isolating a computer resources without the creation of a separate operating system.  Here the two of the main features on Linux that Docker takes advantage of:

1. Namespaces

A namespace is just a folder.  It's a scope.  Within a Linux namespace, software is unaware of processes and software outside of that namespace.  This means that we could have one version of Python installed in one space, and a totally separate version installed in a separate namespace.   

2. CGroups

Now by creating a separate namespace we *are* able to properly partition off sections of our computer and thereby isolate processes and software in each namespace.  But there is still a fixed amount of memory, computing power, hard drive space on each computer.  So while each namespace feels separate, there may still be a struggle over computing resources.  Cgroups prevent this struggle.  A Cgroup limits a namespace to a specific amount of resources, so that one namespace does not hog all of the resources from other namespaces, which otherwise could cause failures in multiple "isolated" environments.    

### A Union FileSystem

The third feature in Linux that Docker takes advantage of is a union filesystem.  Union filesystems were used in the days of CDs.  Imagine that we had chapters of the first Harry Potter book on a CD that we borrowed from our friend, and other chapters of the book in a folder on our computer named `harry_potter`.  Well we could have both the chapters on the CD appear as if they were in the `harry_potter` folder with a union filesystem.  That's one feature of a union filesystem: it allows different folders to act as if it contains files even though the files live somewhere else.

Now imagine that we want to edit a chapter that was on that CD.  If we actually edited the chapter on the CD, our friend would get annoyed, and justifiably.  So instead, we layer our edit on top of the original with a new file and keep the original CD unchanged.  This allows a file system to appear as writable, but without actually changing the file system, also known as copy-on-write. 

This is a union file system.

In fact let's read over the first few paragraphs of [Wikpedia's entry on the subject](https://en.wikipedia.org/wiki/UnionFS):

A union filesystem...

> allows files and directories of separate file systems, known as branches, to be transparently overlaid, forming a single coherent file system. Contents of directories which have the same path within the merged branches will be seen together in a single merged directory, within the new, virtual filesystem.

> When mounting branches, the priority of one branch over the other is specified. So when both branches contain a file with the same name, one gets priority over the other.

> The different branches may be either read-only or read-write file systems, so that writes to the virtual, merged copy are directed to a specific real file system. This allows a file system to appear as writable, but without actually allowing writes to change the file system, also known as copy-on-write. This may be desirable when the media is physically read-only, such as in the case of Live CDs.

This is a really interesting innovation.

By storing files as read-only layers that can be shared between directories, it allows us to reuse a lot of software.  In addition, it becomes very careful about what is unchanged. 

This set of read-only layered files is a docker image.  Now, if we want to interact with this image in a meaningful way, we likely are changing the image somehow.  So when we want to interact with an image by calling `docker run image_name`, docker creates a new writable layer, and this is our container.   

So if we say create three new containers of a `jupyter/scipy` image, each container shares most of the files via the union filesystem, but still has it's own dedicated resources like memory and cpu.  And differences between the files in the containers are in that top new layer.  

So a docker image is a set of read-only layered files.  And a container is a combination of the image along with a this thin writable layer on top of it.

We'll continue to build our understanding of images and containers as we move through Docker, but for now this will do, and we can always come back to this lesson later on.

### Summary

In this lesson, we learned about the underlying technology that allows docker containers to be both lightweight and properly isolated.  Docker makes use of namespaces to prevent mimic an isolated environment, and Cgroups to limit a container's computing resources.  In addition, it uses a union filesystem to share read-only files between containers of the same image.  To make changes to an image, it adds a thin writable layer of the modifications.   

### Resources

[Cgroups and Linux Containers](https://www.youtube.com/watch?v=el7768BNUPw)

[Understanding Docker Internals](https://medium.com/@nagarwal/understanding-the-docker-internals-7ccb052ce9fe)

[Docker Namespace and Cgroups](https://medium.com/@kasunmaduraeng/docker-namespace-and-cgroups-dece27c209c7)

[Docker Containers and Filesystem](https://medium.com/@nagarwal/docker-containers-filesystem-demystified-b6ed8112a04a)